In [1]:
# !pip3 install git+https://github.com/huseinzol05/malaya.git@4.6.1 --no-deps

In [2]:
import malaya
from malaya.preprocessing import Tokenizer
from malaya.text.function import case_of
from malaya.augmentation import (
    replace_similar_consonants, 
    replace_similar_vowels, 
    socialmedia_form,
    vowel_alternate)
from malaya.text import rules
from collections import defaultdict
import random

In [3]:
replace_normalizer = defaultdict(list)
for k, v in rules.rules_normalizer.items():
    if v.count(' ') == 0:
        replace_normalizer[v].append(k)

In [4]:
tokenizer = Tokenizer(duration = False, date = False,
                     ).tokenize

def augment(string):
    
    r = []
    for word in tokenizer(string):
        original_word = word
        word_lower = word.lower()
        try:
            s = socialmedia_form(word_lower)
            if len(s):
                word = case_of(word)(s[0])
            else:
                if word_lower in replace_normalizer and random.random() >= 0.5:
                    word = case_of(word)(random.choice(replace_normalizer[word_lower]))
            
            word = case_of(word)(vowel_alternate(word, 0.9))
            word = case_of(word)(replace_similar_consonants(word, 0.95))
            word = case_of(word)(replace_similar_vowels(word, 0.95))
                
        except Exception as e:
            word = original_word
            pass
        
        r.append(word)
    return ' '.join(r)

In [5]:
string = """
Beliau telah dipilih untuk Parlimen Malaysia pada tahun 1976, pada usia 23 tahun, menggantikan bapanya duduk di kerusi Pekan yang berpangkalan di Pahang. Dari tahun 1982 hingga 1986 beliau menjadi Menteri Besar (Ketua Menteri) Pahang, sebelum memasuki persekutuan Kabinet Tun Dr Mahathir Mohamad pada tahun 1986 sebagai Menteri Kebudayaan, Belia dan Sukan. Beliau telah berkhidmat dalam pelbagai jawatan Kabinet sepanjang baki tahun 1980-an dan 1990-an, termasuk sebagai Menteri Pertahanan dan Menteri Pelajaran. Beliau menjadi Timbalan Perdana Menteri pada 7 Januari 2004, berkhidmat di bawah Perdana Menteri Tun Dato' Seri Abdullah Ahmad Badawi, sebelum menggantikan Badawi setahun selepas Barisan Nasional mengalami kerugian besar dalam pilihan raya 2008. Di bawah kepimpinan beliau, Barisan Nasional memenangi pilihan raya 2013, walaupun buat kali pertama dalam sejarah Malaysia pembangkang memenangi majoriti undi popular.
"""
splitted = malaya.text.function.split_into_sentences(string)
augment(splitted[0])

'Beloau tela dipili utk Parlimen Msia pade tahin 1976 , pade usia 23 tahn , ngegantiin bapanye duduk d kerusi Pelan yg berpangkalan di Pahang .'

In [6]:
splitted[0]

'Beliau telah dipilih untuk Parlimen Malaysia pada tahun 1976, pada usia 23 tahun, menggantikan bapanya duduk di kerusi Pekan yang berpangkalan di Pahang.'

In [7]:
files = ['../pure-text/filtered-dumping-wiki.txt',
        '../pure-text/dumping-cleaned-news.txt']

In [8]:
with open(files[0]) as fopen:
    data = list(filter(None, fopen.read().split('\n')))
    
data = [i for i in data if len(i) >= 2]
len(data)

2037249

In [9]:
from tqdm import tqdm

def loop(strings):
    results = []
    for i in tqdm(range(len(strings))):
        t = ' '.join(tokenizer(strings[i]))
        if random.random() >= 0.3:
            row = augment(strings[i])
            results.append((row, t))
        else:
            results.append((t, t))
    return results

In [11]:
loop(data[:10])

100%|██████████| 10/10 [00:00<00:00, 1039.92it/s]


[('Dork Jan Klaas " Klaas Jan " Untlaar ( lhr 12 August 1983 ) meripakan pemain bole sepak Belande yanckh bermain dhi possi penyerang .',
  'Dirk Jan Klaas " Klaas-Jan " Huntelaar ( lahir 12 Ogos 1983 ) merupakan pemain bola sepak Belanda yang bermain di posisi penyerang .'),
 ('Beliau kini bermain untl lelab Ajax .',
  'Beliau kini bermain untuk kelab Ajax .'),
 ('Hypo Arene .', 'Hypo-Arena .'),
 ('Hypo Arene ( dlu dkenali sbgai ) iala senua stadium serbe gune d Klagenfurt , Austria .',
  'Hypo-Arena ( dahulu dikenali sebagai ) ialah sebuah stadium serba guna di Klagenfurt , Austria .'),
 ('Ia merupakan stadium pasukan Austria Karnten .',
  'Ia merupakan stadium pasukan Austria Karnten .'),
 ('Stadium lame dikenli sbg Wortherseestafion , dibine pade 1960 dn mempunyai kapasiti sebanyak 10,900 .',
  'Stadium lama dikenali sebagai Wortherseestadion , dibina pada 1960 dan mempunyai kapasiti sebanyak 10,900 .'),
 ('Ia dirobohkan pada 2005 dan digantikan dengan Hypo-Arena yang baru , juga d

In [12]:
data = random.sample(data, 700000)

In [13]:
import cleaning

results1 = cleaning.multiprocessing(data, loop)

100%|██████████| 43750/43750 [00:24<00:00, 1800.67it/s]


In [14]:
not_same = 0
for r in tqdm(results1):
    if r[0] != r[1]:
        not_same += 1

not_same / len(results1)

100%|██████████| 700000/700000 [00:00<00:00, 1852324.92it/s]


0.6845657142857143

In [15]:
import json

with open('results-wiki-spelling.json', 'w') as fopen:
    json.dump(results1, fopen)

In [16]:
with open(files[1]) as fopen:
    data = list(filter(None, fopen.read().split('\n')))
    
data = [i for i in data if len(i) >= 2]
len(data)

3483869

In [17]:
data = random.sample(data, 700000)

In [18]:
results1 = cleaning.multiprocessing(data, loop)

100%|██████████| 43750/43750 [00:28<00:00, 1553.36it/s]


In [20]:
not_same = 0
for r in tqdm(results1):
    if r[0] != r[1]:
        not_same += 1

not_same / len(results1)

100%|██████████| 700000/700000 [00:00<00:00, 1865017.29it/s]


0.6917

In [21]:
import json

with open('results-news-spelling.json', 'w') as fopen:
    json.dump(results1, fopen)

In [22]:
results1[:10]

[('Intinye , ia menekankan aturan penggunaan AI arus mempertimbangkan paktor faktut spetti keamaman dan keadolan untuk menyeimbangkan potenso mamfaat n bahaye perkembangan teknologi .',
  'Intinya , ia menekankan aturan penggunaan AI harus mempertimbangkan faktor-faktor seperti keamanan dan keadilan untuk menyeimbangkan potensi manfaat dan bahaya perkembangan teknologi .'),
 ('Mereke mendesainnye mgn teme berbagai musim d Jepang .',
  'Mereka mendesainnya dengan tema berbagai musim di Jepang .'),
 ('Pgo ini , Menteri KPT , Dtuk Seri Idrs Juso mengmmkan lapn lagi universiti awam ( UA ) dan swaste tempatan mndpat Perakuan Status Swaakreftasi Agemso Kelayakn Malysia ( MQE ) , yang memblehkan mreke mengakteditso sendiri program pengjian di universiti masing masinf .',
  'Pagi ini , Menteri KPT , Datuk Seri Idris Jusoh mengumumkan lapan lagi universiti awam ( UA ) dan swasta tempatan mendapat Perakuan Status Swaakreditasi Agensi Kelayakan Malaysia ( MQA ) , yang membolehkan mereka mengakred